In [1]:
# Import the dependencies.

import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.

city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Pangoa,PE,2020-04-07 00:53:22,-12.12,-73.00,73.38,70,6,0.81,clear sky,0.000000,0.000000
1,1,Rikitea,PF,2020-04-07 00:53:22,-23.12,-134.97,76.37,67,81,7.47,broken clouds,0.000000,0.000000
2,2,Cherskiy,RU,2020-04-07 00:53:23,68.75,161.30,18.84,97,100,15.73,light snow,0.000000,0.052362
3,3,Hermanus,ZA,2020-04-07 00:53:23,-34.42,19.23,57.99,66,72,10.00,light rain,0.019291,0.000000
4,4,Cape Town,ZA,2020-04-07 00:50:45,-33.93,18.42,55.99,62,75,3.36,broken clouds,0.000000,0.000000


In [3]:
# Configure gmaps to use the Google API key.

gmaps.configure(api_key = g_key)

In [4]:
# Ask the customer to add a minimum and maximum temperature value.
# Ask the customer to add if they want it to be raining or snowing.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = (input("Do you want it to be raining? (yes/no) "))
snowing = (input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
1,1,Rikitea,PF,2020-04-07 00:53:22,-23.12,-134.97,76.37,67,81,7.47,broken clouds,0.0,0.0
10,10,Butaritari,KI,2020-04-07 00:53:24,3.07,172.79,82.85,70,100,2.46,overcast clouds,0.0,0.0
11,11,Salina,US,2020-04-07 00:52:47,38.84,-97.61,75.20,53,1,17.22,clear sky,0.0,0.0
12,12,Santa Rosa,PH,2020-04-07 00:53:24,14.31,121.11,88.00,66,20,4.70,few clouds,0.0,0.0
15,15,Vila Velha,BR,2020-04-07 00:53:25,-20.33,-40.29,78.80,78,75,6.93,broken clouds,0.0,0.0
21,21,Arraial Do Cabo,BR,2020-04-07 00:53:26,-22.97,-42.03,77.00,83,11,13.87,few clouds,0.0,0.0
22,22,Ambunti,PG,2020-04-07 00:53:26,-4.24,142.84,79.27,82,100,2.33,overcast clouds,0.0,0.0
26,26,Puerto Escondido,MX,2020-04-07 00:53:27,15.85,-97.07,82.40,78,5,9.17,clear sky,0.0,0.0
27,27,Meulaboh,ID,2020-04-07 00:53:27,4.14,96.13,77.29,85,97,5.06,overcast clouds,0.0,0.0
34,34,Muisne,EC,2020-04-07 00:53:28,0.60,-80.03,77.72,79,43,5.12,scattered clouds,0.0,0.0


In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,76.37,broken clouds,-23.12,-134.97,
10,Butaritari,KI,82.85,overcast clouds,3.07,172.79,
11,Salina,US,75.20,clear sky,38.84,-97.61,
12,Santa Rosa,PH,88.00,few clouds,14.31,121.11,
15,Vila Velha,BR,78.80,broken clouds,-20.33,-40.29,
21,Arraial Do Cabo,BR,77.00,few clouds,-22.97,-42.03,
22,Ambunti,PG,79.27,overcast clouds,-4.24,142.84,
26,Puerto Escondido,MX,82.40,clear sky,15.85,-97.07,
27,Meulaboh,ID,77.29,overcast clouds,4.14,96.13,
34,Muisne,EC,77.72,scattered clouds,0.60,-80.03,


In [7]:
# Set parameters to search for a hotel.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Iterate through the DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,76.37,broken clouds,-23.12,-134.97,Pension Maro'i
10,Butaritari,KI,82.85,overcast clouds,3.07,172.79,Isles Sunset Lodge
11,Salina,US,75.20,clear sky,38.84,-97.61,Holiday Inn Express & Suites Salina
12,Santa Rosa,PH,88.00,few clouds,14.31,121.11,Erlyn’s Sta Rosa City Home
15,Vila Velha,BR,78.80,broken clouds,-20.33,-40.29,Hotel Vitória Palace
21,Arraial Do Cabo,BR,77.00,few clouds,-22.97,-42.03,Pousada Porto Praia
22,Ambunti,PG,79.27,overcast clouds,-4.24,142.84,Ambunti Lodge
26,Puerto Escondido,MX,82.40,clear sky,15.85,-97.07,Aldea del Bazar Hotel and Spa
27,Meulaboh,ID,77.29,overcast clouds,4.14,96.13,Kost Pria
34,Muisne,EC,77.72,scattered clouds,0.60,-80.03,Quinta Samay


In [10]:
# Create the output csv file.

output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the city_data into the csv file.
hotel_df.to_csv(output_data_file)

In [11]:
# Add a heatmap of temperature for the vacation spots.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Add a heatmap of temperature for the vacation spots and marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [15]:
# Store the DataFrame Row.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))